In [4]:
# !pip install panda
import random
import pandas as pd

In [7]:
filename = 'creditcard.csv'
data = pd.read_csv(filename)

In [17]:
# print(data.values)
print(len(data.columns))
# X = data.iloc[:,]
numIndex = len(data.columns)
featureVec = data.iloc[:,:numIndex-1]
labelVec = data.iloc[:,numIndex-1]
# print(featureVec.values[4,:])
print(len(labelVec.values))

31
[ 2.00000000e+00 -1.15823309e+00  8.77736755e-01  1.54871785e+00
  4.03033934e-01 -4.07193377e-01  9.59214625e-02  5.92940745e-01
 -2.70532677e-01  8.17739308e-01  7.53074432e-01 -8.22842878e-01
  5.38195550e-01  1.34585159e+00 -1.11966983e+00  1.75121130e-01
 -4.51449183e-01 -2.37033239e-01 -3.81947870e-02  8.03486925e-01
  4.08542360e-01 -9.43069713e-03  7.98278495e-01 -1.37458080e-01
  1.41266984e-01 -2.06009588e-01  5.02292224e-01  2.19422230e-01
  2.15153147e-01  6.99900000e+01]


In [5]:
def load_data(filename = 'creditcard.csv', ignoreTime = False):
    data = pd.read_csv(filename)
    numIndex = len(data.columns)
    featureVec = data.iloc[:,:numIndex-1]
    labelVec = data.iloc[:,numIndex-1]
    return featureVec.values, labelVec.values

creditcard.csv	fraud_detection.ipynb  README.md


In [ ]:
def resampling(X,y,alpha,method='under_sampling'):
    
    